<a href="https://colab.research.google.com/github/Hardik-Garbyal/Spam-Detection/blob/main/FeatureModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the Dependencies**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix

**Data Collection & Pre-Processing**

In [ ]:
# loading the data from csv file to a pandas Dataframe
raw_tweet_data = pd.read_csv('hardik.csv')

In [ ]:
print(raw_tweet_data)

          Id                                              Tweet  following  \
0      10091  It's the everything else that's complicated. #...        0.0   
1      10172  Eren sent a glare towards Mikasa then nodded a...        0.0   
2       7012  I posted a new photo to Facebook http://fb.me/...        0.0   
3       3697  #jan Idiot Chelsea Handler Diagnoses Trump Wit...     3319.0   
4      10740  Pedophile Anthony Weiner is TERRIFIED of Getti...     4840.0   
...      ...                                                ...        ...   
17535   6801  This is the 2nd time we have tried 2 contact u...        NaN   
17536   6802             Will Ã_ b going to esplanade fr home?        NaN   
17537   6803  Pity, * was in mood for that. So...any other s...        NaN   
17538   6804  The guy did some bitching but I acted like i'd...        NaN   
17539   6805                         Rofl. Its true to its name        NaN   

       followers  actions  is_retweet       location     Type  

In [ ]:
# replace the missing values with new values
tweet_data = raw_tweet_data.fillna({
'Id	':0,
'Tweet': "No Tweet",
'following': 0,
'followers': 0,
'actions': 0,
'is_retweet': "Nothing",
'location': "No place",
'Type': "Not determined"
})

In [ ]:
# printing the first 5 rows of the dataframe
tweet_data.head()

,Id,Tweet,following,followers,actions,is_retweet,location,Type
0,10091,It's the everything else that's complicated. #...,0.0,11500.0,0.0,0.0,Chicago,Quality
1,10172,Eren sent a glare towards Mikasa then nodded a...,0.0,0.0,0.0,0.0,No place,Quality
2,7012,I posted a new photo to Facebook http://fb.me/...,0.0,0.0,0.0,0.0,"Scotland, U.K",Quality
3,3697,#jan Idiot Chelsea Handler Diagnoses Trump Wit...,3319.0,611.0,294.0,0.0,"Atlanta, Ga",Spam
4,10740,Pedophile Anthony Weiner is TERRIFIED of Getti...,4840.0,1724.0,1522.0,0.0,Blumberg,Spam


In [ ]:
# checking the number of rows and columns in the dataframe
tweet_data.shape

(17540, 8)

**Label Encoding**

In [ ]:
# label Spam tweet as 0;  Quality tweet as 1;

tweet_data.loc[tweet_data['Type'] == 'Spam', 'Type',] = 0
tweet_data.loc[tweet_data['Type'] == 'Quality', 'Type',] = 1
#These two lines will represent each spam tweet by 0 and each quality tweet by 1

In [ ]:
tweet_data.head()

,Id,Tweet,following,followers,actions,is_retweet,location,Type
0,10091,It's the everything else that's complicated. #...,0.0,11500.0,0.0,0.0,Chicago,1
1,10172,Eren sent a glare towards Mikasa then nodded a...,0.0,0.0,0.0,0.0,No place,1
2,7012,I posted a new photo to Facebook http://fb.me/...,0.0,0.0,0.0,0.0,"Scotland, U.K",1
3,3697,#jan Idiot Chelsea Handler Diagnoses Trump Wit...,3319.0,611.0,294.0,0.0,"Atlanta, Ga",0
4,10740,Pedophile Anthony Weiner is TERRIFIED of Getti...,4840.0,1724.0,1522.0,0.0,Blumberg,0


In [ ]:
# separating the data as texts and label
# The text (Y) represents the Type column, while the label (X) represents all the rest of input columns (features)

#So first, we will declare the input features:
X = tweet_data['Tweet']
Y = tweet_data['Type']

In [ ]:
print(X)

0        It's the everything else that's complicated. #...
1        Eren sent a glare towards Mikasa then nodded a...
2        I posted a new photo to Facebook http://fb.me/...
3        #jan Idiot Chelsea Handler Diagnoses Trump Wit...
4        Pedophile Anthony Weiner is TERRIFIED of Getti...
                               ...                        
17535    This is the 2nd time we have tried 2 contact u...
17536               Will Ã_ b going to esplanade fr home?
17537    Pity, * was in mood for that. So...any other s...
17538    The guy did some bitching but I acted like i'd...
17539                           Rofl. Its true to its name
Name: Tweet, Length: 17540, dtype: object


In [ ]:
print(Y)

0        1
1        1
2        1
3        0
4        0
        ..
17535    0
17536    1
17537    1
17538    1
17539    1
Name: Type, Length: 17540, dtype: object


**Splitting Data**

**# Now, it is important in machine learning to split the data into training and testing data, in order to train the model and evaluate it.**

In [ ]:
#Using the imported function (train_test_split)
#We gave the testing data 20% and 80% for the training data

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [ ]:
print('The whole ratio: ', X.shape)
print('80% (train data):', X_train.shape)
print('20% (test data): ', X_test.shape)

The whole ratio:  (17540,)
80% (train data): (14032,)
20% (test data):  (3508,)


**Feature Extraction**

This part converts all the text input columns to meaning numerical values, to be understood by "LogisticRegression" function

In [ ]:
# import matplotlib.pyplot as plt
# from PIL import Image
# image_path = '/content/Screenshot 2023-05-22 113722.png'
# image = Image.open(image_path)
# plt.imshow(image)
# plt.axis('off')
# plt.show()

In [ ]:
#  transform a collection of text documents into a matrix representation where each row corresponds to a document, and each column represents a word (feature) with its TF-IDF value.
feature_extraction = TfidfVectorizer(min_df = 1, stop_words= 'english', lowercase=True)
#mind_df - minimum frequency threshold of a word to be include in vocabulary
#stop_words -  words to be excluded from the vocabulary.

In [ ]:
feature_extraction

TfidfVectorizer(stop_words='english')

In [ ]:
# fitting the vectorizer on the training data to learn the vocabulary and TF-IDF weights, and then transforming the training data into a TF-IDF feature matrix.
X_train_features = feature_extraction.fit_transform(X_train)
#After fitting the training data into the vectorizer function, we don't need to fit the testing data, we just use them in the same vecotrizer
X_test_features = feature_extraction.transform(X_test)
# convert Y_train and Y_test values as integers to can be understood by the machine
#They were declared as object

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [ ]:
print('This is the data of X_train before converting to numerical values \n')
print(X_train)

This is the data of X_train before converting to numerical values 

4184     Eastern Illinois University attempts to boost ...
16089    Doc prescribed me morphine cause the other pai...
17237    If you don't, your prize will go to another cu...
14650    I got a call from a landline number. . . I am ...
14167               Not sure I have the stomach for it ...
                               ...                        
6400     Pedestrian hit and killed by train in Lower Va...
15288                               Yo im right by yo work
11513    The car is so fed up with Ron and Harry. Throw...
1688     Truck attacker kills 84 celebrating France's B...
5994     Wojo: Skid raises questions about Ausmus futur...
Name: Tweet, Length: 14032, dtype: object


In [ ]:
print('This is the data of X_train after converting to numerical values \n')
print(X_train_features)

This is the data of X_train after converting to numerical values 

  (0, 20409)	0.4250507045145944
  (0, 14720)	0.1033220121406513
  (0, 11107)	0.40756461647867376
  (0, 6266)	0.341131384464753
  (0, 4897)	0.3855347509007131
  (0, 29867)	0.31972204757047384
  (0, 15204)	0.37102404940372485
  (0, 10636)	0.37102404940372485
  (1, 17422)	0.18722232840241296
  (1, 29330)	0.20849548141925167
  (1, 13338)	0.19991127913598877
  (1, 11793)	0.2383627812363449
  (1, 16877)	0.25555803926306603
  (1, 18961)	0.28779057039896627
  (1, 6522)	0.22463494550777227
  (1, 19580)	0.22384303898664637
  (1, 30805)	0.21212179780159862
  (1, 4644)	0.2623815941985481
  (1, 18987)	0.2827247889982836
  (1, 21754)	0.23176823440805083
  (1, 7242)	0.22384303898664637
  (1, 19683)	0.30111517983683483
  (1, 22947)	0.3238937501237192
  (1, 10074)	0.28779057039896627
  (2, 6798)	0.24484118844309855
  :	:
  (14029, 10345)	0.2707377834023401
  (14029, 17775)	0.1975566890974422
  (14029, 7099)	0.4404088002231429
  (14030, 

**Training the models**

**1) Logistic Regression**

In [ ]:
LR_model = LogisticRegression()

In [ ]:
# training the Logistic Regression model with the training data
LR_model.fit(X_train_features, Y_train)

LogisticRegression()

In [ ]:
import numpy as np

def fgsm_attack(model, X, y, epsilon):
    # Calculate the gradient of the loss with respect to the input
    scores = model.decision_function(X)
    gradient = np.sign(scores)

    # Generate the perturbed examples
    perturbed_X = X + epsilon * gradient[:, np.newaxis]

    return perturbed_X

In [ ]:
epsilon = 0.1  # Adjust the epsilon value to control the strength of the attack
perturbed_X_test = fgsm_attack(LR_model, X_test_features, Y_test, epsilon)


**2) Decision Tree**

In [ ]:
dt_model = DecisionTreeClassifier(random_state=10)


In [ ]:
dt_model.fit(X_train_features, Y_train)

DecisionTreeClassifier(random_state=10)

**Evaluating the trained model**

**1) Logistic Regression**

In [ ]:
# prediction on training data

prediction_on_training_data = LR_model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

In [ ]:
print('Accuracy on training data : ', accuracy_on_training_data)

Accuracy on training data :  0.9561716077537058


In [ ]:
# prediction on test data

prediction_on_test_data = LR_model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

In [ ]:
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on test data :  0.9230330672748005


**2) Decision Tree**

In [ ]:
# prediction on training data
prediction_on_training_data = dt_model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

In [ ]:
print('Accuracy on training data : ', accuracy_on_training_data)

Accuracy on training data :  1.0


In [ ]:
# prediction on test data

prediction_on_test_data = dt_model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

In [ ]:
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on test data :  0.9210376282782212


**The output of the best classifiers**

**1) Logistic Regression**

In [ ]:
input_tweet = ["I just keep holding you down"]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input_tweet)

# making prediction

prediction = LR_model.predict(input_data_features)
print(prediction)


if (prediction[0]==1):
  print('Quality tweet')

else:
  print('Spam tweet')

[1]
Quality tweet


In [ ]:
input_tweet = ["Black slave"]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input_tweet)

# making prediction

prediction = LR_model.predict(input_data_features)
print(prediction)


if (prediction[0]==1):
  print('Quality tweet')

else:
  print('Spam tweet')

[0]
Spam tweet


**2) Decision Tree**

In [ ]:
input_tweet = ["I just keep holding you down"]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input_tweet)
print(input_data_features)
# making prediction

prediction = dt_model.predict(input_data_features)
print(prediction)


if (prediction[0]==1):
  print('Quality tweet')

else:
  print('Spam tweet')

  (0, 16508)	0.4610489050144313
  (0, 14490)	0.8873747276010253
[1]
Quality tweet


In [ ]:
input_tweet = ["Black slave"]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input_tweet)

# making prediction

prediction = dt_model.predict(input_data_features)
print(prediction)


if (prediction[0]==1):
  print('Quality tweet')

else:
  print('Spam tweet')

[1]
Quality tweet


In [ ]:
import tensorflow as tf
import numpy as np

def fgsm_attack(model, input_data, epsilon):
    # Convert the input data into a TensorFlow tensor
    input_data = tf.convert_to_tensor(input_data, dtype=tf.float32)

    # Set the model to be trainable, as we will be computing gradients
    model.trainable = True

    with tf.GradientTape() as tape:
        tape.watch(input_data)
        prediction = model(input_data)
        target_label = tf.argmax(prediction, axis=1)

    # Calculate the loss between the target label and the model's prediction
    loss = tf.keras.losses.sparse_categorical_crossentropy(target_label, prediction)

    # Compute the gradient of the loss with respect to the input data
    gradient = tape.gradient(loss, input_data)

    # Compute the perturbation using the sign of the gradient
    perturbation = epsilon * tf.sign(gradient)

    # Create the adversarial example by adding the perturbation to the input data
    adversarial_example = input_data + perturbation

    # Clip the adversarial example to ensure it stays within a valid range (e.g., pixel values between 0 and 1)
    adversarial_example = tf.clip_by_value(adversarial_example, 0, 1)

    return adversarial_example.numpy()

# Example usage:
# Assuming 'model' is your pre-trained TensorFlow model
# 'input_data' is a single input example (e.g., an image, word embeddings, etc.)
# 'epsilon' is the magnitude of perturbation allowed (a small value, e.g., 0.01)

# Generate the adversarial example using FGSM

